In [1]:
%%configure -f
{"driverMemory": "47696M", 
 "executorCores": 4,
 "executorMemory": "47696M",
 "conf": {"spark.default.parallelism": 1000,
          "spark.driver.maxResultSize": "23848M",
          "spark.sql.shuffle.partitions": 1000
         }
}

In [2]:
model_path = "s3://onai-ml-dev-eu-west-1/company2vec/model/svd_desc_only"
word2id_path = "s3://onai-ml-dev-eu-west-1/company2vec/common"
data_path = "s3://onai-ml-dev-eu-west-1/company2vec/data_desc_only"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1568367294465_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
from gensim.utils import tokenize
from langdetect import detect
import string
from itertools import islice
import numpy as np
from scipy.sparse import csr_matrix
import smart_open
import scipy

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

from pyspark.ml.linalg import Vectors, VectorUDT

import pickle

import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Train a Truncated SVD on the (sparse) company feature matrix

In [4]:
companies_data = spark.read.load("s3://ai-data-lake-dev-eu-west-1/business/capiq/company_denormalized")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
dups = [row.company_id for row in 
         companies_data
         .groupBy("company_id")
         .agg(F.count("*").alias("count"))
         .filter(F.col("count") > 1)
         .collect()
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
companies_data = companies_data.filter(~F.col("company_id").isin(dups))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
max_desc = 1000
cap_description_udf = F.udf(lambda desc: desc[:max_desc])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
companies_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- one_id: string (nullable = true)
 |-- company_id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- number_of_employees: integer (nullable = true)
 |-- latest_revenue: double (nullable = true)
 |-- latest_revenue_growth: double (nullable = true)
 |-- latest_ebitda: double (nullable = true)
 |-- latest_ebitda_margin: double (nullable = true)
 |-- competitor_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- simple_industry_id: string (nullable = true)
 |-- simple_industry: string (nullable = true)
 |-- sic_code: string (nullable = true)
 |-- sic_code_desc: string (nullable = true)
 |-- naics_code: string (nullable = true)
 |-- naics_code_des

In [9]:
x_joined = (spark.read.load(f"{data_path}/raw_company_features")
            .join(companies_data,
                  on = ["company_id"]
                 )
            .select("company_id",
                    cap_description_udf("company_name").alias("company_name"),
                    cap_description_udf("company_description").alias("company_description"),
                    "country",
                    "company_type",
                    "region",
                    "sic_code",
                    "sic_code_desc",
                    "naics_code",
                    "naics_code_desc",
                    "latest_revenue",
                    "latest_revenue_growth",
                    "latest_ebitda",
                    "latest_ebitda_margin",
                    "number_of_employees",
                    "size",
                    "feature_indices",
                    "feature_values"
                   )
           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
x_joined.filter("company_id == '704634'").show(20, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
x_joined.write.parquet(f"{data_path}/company_features_joined", mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
x_joined = (
    spark.read.load(f"{data_path}/company_features_joined")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
x_joined.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- company_id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- company_description: string (nullable = true)
 |-- country: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- region: string (nullable = true)
 |-- sic_code: string (nullable = true)
 |-- sic_code_desc: string (nullable = true)
 |-- naics_code: string (nullable = true)
 |-- naics_code_desc: string (nullable = true)
 |-- latest_revenue: double (nullable = true)
 |-- latest_revenue_growth: double (nullable = true)
 |-- latest_ebitda: double (nullable = true)
 |-- latest_ebitda_margin: double (nullable = true)
 |-- number_of_employees: integer (nullable = true)
 |-- size: integer (nullable = true)
 |-- feature_indices: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- feature_values: array (nullable = true)
 |    |-- element: double (containsNull = true)

In [65]:
selected_cols = ["company_id",
        "company_name",
        "company_description",
        "company_type",
        "country",
        "region",
        "sic_code",
        "sic_code_desc",
        "naics_code",
        "naics_code_desc",
        "latest_revenue",
        "latest_revenue_growth",
        "latest_ebitda",
        "latest_ebitda_margin",
        "number_of_employees"
       ]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
def zipped_data(x_rdd):
    ret = []
    n_rows = 0
    n_cols = None
    
    def generate_sparse_matrix(x_rdd):
        nonlocal n_cols
        nonlocal n_rows
        for i,el in enumerate(x_rdd.collect()):
            ret.append([el[col] for col in selected_cols])
            if n_cols is None:
                n_cols = el.size
            for j,v in zip(el.feature_indices, el.feature_values):
                yield (i,j,v)
            n_rows += 1

    row_ind, col_ind, data = zip(*generate_sparse_matrix(x_rdd))

    return ret, (n_rows, n_cols), (row_ind, col_ind, data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
metadata, shape, (row_ind, col_ind, data) = zipped_data(x_joined)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
X = csr_matrix((data, (row_ind, col_ind)), shape=shape)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
X.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3385668, 42848)

In [83]:
len(metadata)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3385668

In [84]:
X_metadata = pd.DataFrame(metadata, columns=selected_cols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
X_metadata.to_parquet(f"{model_path}/X/company_info.pd")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
with smart_open.open(f"{model_path}/X/X.npz", "wb") as f:
    scipy.sparse.save_npz(f, X)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=200, n_iter=20, random_state=42)
svd.fit(X)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TruncatedSVD(algorithm='randomized', n_components=200, n_iter=20,
             random_state=42, tol=0.0)

In [88]:
V = svd.components_

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
V.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(200, 42848)

In [90]:
with smart_open.open(f"{model_path}/V/mat.np", "wb") as f:
    np.save(f, V, allow_pickle=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
with smart_open.open(f"{model_path}/truncated_svd.pkl", "wb") as f:
    pickle.dump(svd, f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
with smart_open.open(f"{model_path}/truncated_svd.pkl", "rb") as f:
    svd = pickle.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
with smart_open.open(f"{model_path}/V/mat.np", "rb") as f:
    V = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Now that we have SVD, project the data and write associated descriptions/metadata

In [94]:
X_pca = X.dot(V.T)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
company2id_ini = {}
for i,desc in X_metadata.loc[:, ["company_id"]].iterrows():
    company2id_ini[desc["company_id"]] = i

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
len(company2id_ini)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3385668

In [107]:
query_company = 291847304

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
company2id_ini[query_company]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1960472

In [130]:
idx = X_metadata.loc[:, ["company_id"]].values == query_company

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
X_pca[idx.reshape((idx.shape[0],)),:]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 6.90798509e+00,  5.04505937e+00, -5.59916004e+00,
        -1.75589511e+00,  1.89862666e-01,  1.52853898e+00,
        -7.11982686e-01, -1.41671441e+00,  2.23851210e-01,
         9.38299269e-01, -1.26336157e-01, -9.49901763e-02,
         3.90304398e-01,  1.04732130e+00, -3.22690494e+00,
        -5.68173120e-01, -1.02766119e+00, -3.79361572e-01,
         1.47866222e-01, -6.12579554e-02, -2.17371705e-01,
         6.02918052e-02,  2.18251510e-02, -6.61606257e-01,
         1.04184537e+00,  3.74542734e-01,  5.42752907e-02,
         1.64415705e+00,  6.38461905e-01,  1.49945838e+00,
        -3.02101310e+00,  9.98106487e-01, -4.41823323e-01,
         2.47924747e-02, -1.25629798e+00,  2.57662489e+00,
        -7.42644835e-02, -3.41478293e-01, -4.96224328e-01,
        -4.14216923e-01, -4.84869564e-01,  5.88810866e-01,
         6.68464847e-01, -1.67071430e+00, -3.53384142e-02,
         9.48045529e-01, -1.23402956e+00, -2.64081896e+00,
         1.36825670e+00,  5.33840895e-01,  1.06989218e-0

In [132]:
X_pca = X_pca.astype(np.float32)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
with smart_open.open(f"{model_path}/X/X_pca.np", "wb") as f:
    np.save(f, X_pca, allow_pickle=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
with smart_open.open(f"{model_path}/X/x_pca.np", "wb") as f:
    np.save(f, X_pca[:10000], allow_pickle=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…